<a href="https://colab.research.google.com/github/correctchemist/code/blob/main/myway_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit-pypi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.5 MB 1.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-11-14 08:19:58--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   354MB/s    in 0.2s    

2022-11-14 08:19:59 (354 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [4]:
# https://www.rdkit.org/
#https://github.com/rdkit/rdkit
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# https://pandas.pydata.org
import pandas as pd

# https://numpy.org/doc/stable/release.html
import numpy as np

In [5]:
# import session_info
# session_info.show()

In [6]:
dataset = pd.read_csv('bioactivity_preprocessed_data1.csv')

In [7]:
dataset.head()

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0


In [8]:
# There might be one or more valid SMILES that can represent one compound
# Thanks to Pat Walters for this information,checkout his excellent blog: https://www.blogger.com/profile/18223198920629617711
def canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles] 
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

In [9]:
# Canonical SMILES
Canon_SMILES = canonical_smiles(dataset.SMILES)
len(Canon_SMILES)

602

In [10]:
# Put the smiles in the dataframe
dataset['SMILES'] = Canon_SMILES
dataset

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0
...,...,...,...,...
597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0
600,CHEMBL1956285,Cc1cc(Nc2ccc(S(F)(F)(F)(F)F)cc2)n2nc(C(C)(F)F)...,active,10.0


In [11]:
# Create a list for duplicate smiles
duplicates_smiles = dataset[dataset['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)

51

In [12]:
# Create a list for duplicate smiles
dataset[dataset['SMILES'].isin(duplicates_smiles)].sort_values(by=['SMILES'])

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
97,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,190100.00
419,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,intermediate,3900.00
7,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,190100.00
326,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,190546.07
304,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,11390.00
...,...,...,...,...
572,CHEMBL2012827,O=C(NC1CCCC1)c1ccc(-n2cnc3ccccc32)s1,intermediate,2506.00
333,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,154881.66
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.00
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.00


In [13]:
dataset_new = dataset.drop_duplicates(subset=['SMILES'])
len(dataset_new)

551

In [14]:
dataset_new

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0
...,...,...,...,...
596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,250000.0
597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0


In [18]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names 

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(dataset_new['SMILES'])

In [19]:
df_with_200_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_200_descriptors

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.744296,-3.693263,13.744296,0.327769,0.774008,331.371,314.235,331.120843,124,0,...,0,0,0,0,0,0,0,0,0,0
1,13.252877,-1.606550,13.252877,0.164514,0.710808,370.202,358.106,369.000055,114,0,...,0,0,0,0,0,0,0,0,0,0
2,13.727252,-3.617987,13.727252,0.218218,0.719488,384.229,370.117,383.015705,120,0,...,0,0,0,0,0,0,0,0,0,0
3,13.269921,-1.677155,13.269921,0.265269,0.753503,317.344,302.224,317.105193,118,0,...,0,0,0,0,0,0,0,0,0,0
4,13.699475,-3.640487,13.699475,0.278290,0.800108,305.333,290.213,305.105193,114,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,12.345880,-3.603915,12.345880,0.208836,0.874182,276.292,260.164,276.111007,106,0,...,0,0,0,0,0,0,0,0,0,0
547,12.561833,-3.705638,12.561833,0.243275,0.850289,302.330,284.186,302.126657,116,0,...,0,0,0,0,0,0,0,0,0,0
548,12.701226,-3.662697,12.701226,0.206424,0.715545,338.363,320.219,338.126657,128,0,...,0,0,0,0,0,0,0,0,0,0
549,12.240651,-3.423406,12.240651,0.052663,0.835805,247.254,234.150,247.095691,94,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('bioactivity_preprocessed_data2.csv')
df

,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0
...,...,...,...,...,...
546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,250000.0
547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0


In [22]:
df_no_smiles = df.drop(columns='canonical_smiles')

In [23]:
smiles = []

for i in df.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [24]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

,Unnamed: 0,molecule_chembl_id,bioactivity_class,standard_value,canonical_smiles
0,0,CHEMBL199572,inactive,42600.0,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O
1,1,CHEMBL199574,inactive,142600.0,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1
2,2,CHEMBL372561,inactive,93400.0,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O
3,3,CHEMBL370865,inactive,153500.0,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1
4,4,CHEMBL199575,inactive,200000.0,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O
...,...,...,...,...,...
546,596,CHEMBL4544292,inactive,250000.0,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O
547,597,CHEMBL4569109,inactive,250000.0,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1
548,598,CHEMBL4568957,inactive,250000.0,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1
549,599,CHEMBL4449622,inactive,250000.0,Cn1nc(O)c(C(N)=O)c1COc1ccccc1


In [25]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [27]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [28]:
df_lipinski = lipinski(df_clean_smiles.canonical_smiles)
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,331.371,4.32840,1.0,2.0
1,370.202,4.55280,2.0,2.0
2,384.229,4.57710,1.0,2.0
3,317.344,4.30410,2.0,2.0
4,305.333,3.81460,1.0,2.0
...,...,...,...,...
546,276.292,2.09600,1.0,5.0
547,302.330,2.48610,1.0,5.0
548,338.363,3.27630,1.0,5.0
549,247.254,0.80360,2.0,5.0


In [30]:
df

,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0
...,...,...,...,...,...
546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,250000.0
547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0


In [31]:
df_combined = pd.concat([df,df_lipinski], axis=1)

In [32]:
df_combined

,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value,MW,LogP,NumHDonors,NumHAcceptors
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0,331.371,4.32840,1.0,2.0
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0,370.202,4.55280,2.0,2.0
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0,384.229,4.57710,1.0,2.0
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0,317.344,4.30410,2.0,2.0
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0,305.333,3.81460,1.0,2.0
...,...,...,...,...,...,...,...,...,...
546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,250000.0,276.292,2.09600,1.0,5.0
547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0,302.330,2.48610,1.0,5.0
548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0,338.363,3.27630,1.0,5.0
549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0,247.254,0.80360,2.0,5.0


In [33]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [34]:
df_combined.standard_value.describe()

count       551.000000
mean      33974.369383
std       82621.937247
min           6.000000
25%         327.000000
50%        7000.000000
75%       27490.000000
max      870963.590000
Name: standard_value, dtype: float64

In [35]:
-np.log10( (10**-9)* 100000000 )

1.0

In [36]:
-np.log10( (10**-9)* 10000000000 )

-1.0

In [37]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [38]:
df_norm = norm_value(df_combined)
df_norm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,42600.0
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,142600.0
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,93400.0
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,153500.0
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,200000.0
...,...,...,...,...,...,...,...,...,...
546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,250000.0
547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,250000.0
548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,250000.0
549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,250000.0


In [39]:
df_norm = norm_value(df_combined)
df_norm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,42600.0
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,142600.0
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,93400.0
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,153500.0
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,200000.0
...,...,...,...,...,...,...,...,...,...
546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,250000.0
547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,250000.0
548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,250000.0
549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,250000.0


In [40]:
df_norm.standard_value_norm.describe()

count       551.000000
mean      33974.369383
std       82621.937247
min           6.000000
25%         327.000000
50%        7000.000000
75%       27490.000000
max      870963.590000
Name: standard_value_norm, dtype: float64

In [41]:
df_final = pIC50(df_norm)
df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,4.370590
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,3.845880
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,4.029653
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,3.813892
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,3.698970
...,...,...,...,...,...,...,...,...,...
546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,3.602060
547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,3.602060
548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,3.602060
549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,3.602060


In [42]:
df_final.pIC50.describe()

count    551.000000
mean       5.494939
std        1.165906
min        3.060000
25%        4.560849
50%        5.154902
75%        6.485471
max        8.221849
Name: pIC50, dtype: float64

In [43]:
df_final.to_csv('dihydoorotate_04_bioactivity_data2_3class_pIC50.csv')

In [44]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

--2022-11-14 09:12:17--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2022-11-14 09:12:18--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  --.-KB/s    in 0.07s   

2022-11-14 09:12:18 (367 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2022-11-14 09:12:18--  https://github

In [45]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

In [46]:
import pandas as pd

In [47]:
df3 = pd.read_csv('dihydoorotate_04_bioactivity_data2_3class_pIC50.csv')

In [48]:
df3

,Unnamed: 0,Unnamed: 0.1,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,4.370590
1,1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,3.845880
2,2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,4.029653
3,3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,3.813892
4,4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,3.698970
...,...,...,...,...,...,...,...,...,...,...
546,546,596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,3.602060
547,547,597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,3.602060
548,548,598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,3.602060
549,549,599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,3.602060


In [50]:
df3 = df3.drop(columns=['Unnamed: 0.1'])
df3

,Unnamed: 0,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,4.370590
1,1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,3.845880
2,2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,4.029653
3,3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,3.813892
4,4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,3.698970
...,...,...,...,...,...,...,...,...,...
546,546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,3.602060
547,547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,3.602060
548,548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,3.602060
549,549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,3.602060


In [51]:
df3 = df3.drop(columns=['Unnamed: 0'])
df3

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,331.371,4.32840,1.0,2.0,4.370590
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,370.202,4.55280,2.0,2.0,3.845880
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,384.229,4.57710,1.0,2.0,4.029653
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,317.344,4.30410,2.0,2.0,3.813892
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,305.333,3.81460,1.0,2.0,3.698970
...,...,...,...,...,...,...,...,...
546,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,276.292,2.09600,1.0,5.0,3.602060
547,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,302.330,2.48610,1.0,5.0,3.602060
548,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,338.363,3.27630,1.0,5.0,3.602060
549,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,247.254,0.80360,2.0,5.0,3.602060


In [52]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [53]:
! cat molecule.smi | head -5

CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O	CHEMBL199572
O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1	CHEMBL199574
CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O	CHEMBL372561
O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1	CHEMBL370865
CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O	CHEMBL199575


In [54]:
! cat molecule.smi | wc -l

551


In [55]:
! cat padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv


In [56]:
! bash padel.sh

Processing CHEMBL199574 in molecule.smi (2/551). 
Processing CHEMBL199572 in molecule.smi (1/551). 
Processing CHEMBL372561 in molecule.smi (3/551). Average speed: 2.20 s/mol.
Processing CHEMBL370865 in molecule.smi (4/551). Average speed: 1.16 s/mol.
Processing CHEMBL199575 in molecule.smi (5/551). Average speed: 1.05 s/mol.
Processing CHEMBL218467 in molecule.smi (7/551). Average speed: 0.85 s/mol.
Processing CHEMBL200536 in molecule.smi (6/551). Average speed: 0.87 s/mol.
Processing CHEMBL973 in molecule.smi (8/551). Average speed: 0.73 s/mol.
Processing CHEMBL217893 in molecule.smi (9/551). Average speed: 0.67 s/mol.
Processing CHEMBL217897 in molecule.smi (11/551). Average speed: 0.57 s/mol.
Processing CHEMBL217951 in molecule.smi (10/551). Average speed: 0.64 s/mol.
Processing CHEMBL387254 in molecule.smi (12/551). Average speed: 0.55 s/mol.
Processing CHEMBL373493 in molecule.smi (13/551). Average speed: 0.51 s/mol.
Processing CHEMBL374338 in molecule.smi (14/551). Average speed

In [57]:
! ls -l

total 109400
-rw-r--r-- 1 root root    40373 Nov 14 08:16 bioactivity_preprocessed_data1.csv
-rw-r--r-- 1 root root    39179 Nov 14 08:45 bioactivity_preprocessed_data2.csv
-rw-r--r-- 1 root root   990914 Nov 14 09:22 descriptors_output.csv
-rw-r--r-- 1 root root    68319 Nov 14 09:10 dihydoorotate_04_bioactivity_data2_3class_pIC50.csv
drwx------ 6 root root     4096 Nov 14 08:19 drive
drwxr-xr-x 3 root root     4096 Nov 14 09:12 __MACOSX
-rwxr-xr-x 1 root root 85055499 Mar 11  2020 Miniconda3-py37_4.8.2-Linux-x86_64.sh
-rw-r--r-- 1 root root    28717 Nov 14 09:19 molecule.smi
drwxrwxr-x 4 root root     4096 May 30  2020 PaDEL-Descriptor
-rw-r--r-- 1 root root      231 Nov 14 09:12 padel.sh
-rw-r--r-- 1 root root 25768637 Nov 14 09:12 padel.zip
drwxr-xr-x 1 root root     4096 Nov 10 14:30 sample_data


In [58]:
df3_X = pd.read_csv('descriptors_output.csv')

In [59]:
df3_X

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,CHEMBL199574,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL199572,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL372561,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL370865,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL199575,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,CHEMBL4544292,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
547,CHEMBL4569109,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
548,CHEMBL4449622,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
549,CHEMBL4568957,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
df3_X = pd.concat([df_with_200_descriptors,df3_X], axis=1)
df3_X

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,13.744296,-3.693263,13.744296,0.327769,0.774008,331.371,314.235,331.120843,124,0,...,0,0,0,0,0,0,0,0,0,0
1,13.252877,-1.606550,13.252877,0.164514,0.710808,370.202,358.106,369.000055,114,0,...,0,0,0,0,0,0,0,0,0,0
2,13.727252,-3.617987,13.727252,0.218218,0.719488,384.229,370.117,383.015705,120,0,...,0,0,0,0,0,0,0,0,0,0
3,13.269921,-1.677155,13.269921,0.265269,0.753503,317.344,302.224,317.105193,118,0,...,0,0,0,0,0,0,0,0,0,0
4,13.699475,-3.640487,13.699475,0.278290,0.800108,305.333,290.213,305.105193,114,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,12.345880,-3.603915,12.345880,0.208836,0.874182,276.292,260.164,276.111007,106,0,...,0,0,0,0,0,0,0,0,0,0
547,12.561833,-3.705638,12.561833,0.243275,0.850289,302.330,284.186,302.126657,116,0,...,0,0,0,0,0,0,0,0,0,0
548,12.701226,-3.662697,12.701226,0.206424,0.715545,338.363,320.219,338.126657,128,0,...,0,0,0,0,0,0,0,0,0,0
549,12.240651,-3.423406,12.240651,0.052663,0.835805,247.254,234.150,247.095691,94,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df3_Y = df3['pIC50']
df3_Y

0      4.370590
1      3.845880
2      4.029653
3      3.813892
4      3.698970
         ...   
546    3.602060
547    3.602060
548    3.602060
549    3.602060
550    8.000000
Name: pIC50, Length: 551, dtype: float64

In [62]:
dataset3 = pd.concat([df3_X,df3_Y], axis=1)
dataset3

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,13.744296,-3.693263,13.744296,0.327769,0.774008,331.371,314.235,331.120843,124,0,...,0,0,0,0,0,0,0,0,0,4.370590
1,13.252877,-1.606550,13.252877,0.164514,0.710808,370.202,358.106,369.000055,114,0,...,0,0,0,0,0,0,0,0,0,3.845880
2,13.727252,-3.617987,13.727252,0.218218,0.719488,384.229,370.117,383.015705,120,0,...,0,0,0,0,0,0,0,0,0,4.029653
3,13.269921,-1.677155,13.269921,0.265269,0.753503,317.344,302.224,317.105193,118,0,...,0,0,0,0,0,0,0,0,0,3.813892
4,13.699475,-3.640487,13.699475,0.278290,0.800108,305.333,290.213,305.105193,114,0,...,0,0,0,0,0,0,0,0,0,3.698970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,12.345880,-3.603915,12.345880,0.208836,0.874182,276.292,260.164,276.111007,106,0,...,0,0,0,0,0,0,0,0,0,3.602060
547,12.561833,-3.705638,12.561833,0.243275,0.850289,302.330,284.186,302.126657,116,0,...,0,0,0,0,0,0,0,0,0,3.602060
548,12.701226,-3.662697,12.701226,0.206424,0.715545,338.363,320.219,338.126657,128,0,...,0,0,0,0,0,0,0,0,0,3.602060
549,12.240651,-3.423406,12.240651,0.052663,0.835805,247.254,234.150,247.095691,94,0,...,0,0,0,0,0,0,0,0,0,3.602060


In [63]:
dataset3.to_csv('dihydroorotate_06_bioactivity_data2_3class_pIC50_pubchem_fp.csv', index=False)